In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests

from bokeh.plotting import figure, output_file, show, curdoc
from bokeh.models import ColumnDataSource, Legend, GeoJSONDataSource, LinearColorMapper, ColorBar, Range1d
from bokeh.models import NumeralTickFormatter, HoverTool, LabelSet, Panel, Tabs, Slider, CustomJS, TapTool, CDSView
from bokeh.models.widgets import TableColumn, DataTable, NumberFormatter, Dropdown, Select, RadioButtonGroup, TableColumn
from bokeh.palettes import Category20c, Spectral, GnBu, brewer, PRGn, RdYlGn
from bokeh.io import curdoc, output_notebook, show, output_file
from bokeh.layouts import row, column, gridplot
from bokeh.palettes import Viridis6 as palette
from bokeh.transform import cumsum

import colorcet

import geopandas as gpd
import pycountry
import json

import warnings
warnings.filterwarnings('ignore')

from rich.console import Console

In [2]:
df = pd.read_csv('global_power_plant_database_v_1_3/global_power_plant_database.csv', low_memory=False)

In [3]:
df.sample(5).T

,19709,29728,5962,28612,21625
country,RUS,USA,CHN,USA,THA
country_long,Russia,United States of America,China,United States of America,Thailand
name,Zagorskaya PSPP,Long Beach Generation LLC,GoSP} AM,High Point POLO,Tha Thung
gppd_idnr,WRI1003865,USA0000341,WKS0068522,USA0057196,WRI1000180
capacity_mw,1200.0,252.0,30.0,1.8,36.0
latitude,56.4822,33.7641,36.397,35.97,15.2063
longitude,38.1856,-118.2248,95.262,-79.96,105.4292
primary_fuel,Hydro,Gas,Solar,Oil,Hydro
other_fuel1,NaN,NaN,NaN,NaN,NaN
other_fuel2,NaN,NaN,NaN,NaN,NaN


In [4]:
df.shape

(34936, 36)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34936 entries, 0 to 34935
Data columns (total 36 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   country                         34936 non-null  object 
 1   country_long                    34936 non-null  object 
 2   name                            34936 non-null  object 
 3   gppd_idnr                       34936 non-null  object 
 4   capacity_mw                     34936 non-null  float64
 5   latitude                        34936 non-null  float64
 6   longitude                       34936 non-null  float64
 7   primary_fuel                    34936 non-null  object 
 8   other_fuel1                     1944 non-null   object 
 9   other_fuel2                     276 non-null    object 
 10  other_fuel3                     92 non-null     object 
 11  commissioning_year              17447 non-null  float64
 12  owner                           

In [6]:
df.query('country_long=="India"').info()

<class 'pandas.core.frame.DataFrame'>
Index: 1589 entries, 14052 to 15640
Data columns (total 36 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   country                         1589 non-null   object 
 1   country_long                    1589 non-null   object 
 2   name                            1589 non-null   object 
 3   gppd_idnr                       1589 non-null   object 
 4   capacity_mw                     1589 non-null   float64
 5   latitude                        1589 non-null   float64
 6   longitude                       1589 non-null   float64
 7   primary_fuel                    1589 non-null   object 
 8   other_fuel1                     193 non-null    object 
 9   other_fuel2                     1 non-null      object 
 10  other_fuel3                     0 non-null      object 
 11  commissioning_year              496 non-null    float64
 12  owner                           32

In [7]:
df.query('country_long=="India"').estimated_generation_gwh_2017.sum()

1434479.65

In [8]:
df.query('country_long=="India"').generation_gwh_2017.sum()

1115205.1145344903

In [9]:
df[df['generation_gwh_2017'].isna()].query('country_long=="India"')[['generation_gwh_2017', 'estimated_generation_gwh_2017']]

,generation_gwh_2017,estimated_generation_gwh_2017
14052,NaN,4.29
14053,NaN,516.25
14054,NaN,56.88
14065,NaN,44.25
14068,NaN,3160.77
...,...,...
15636,NaN,117.52
15637,NaN,421.43
15638,NaN,114.51
15639,NaN,20.30


In [10]:
df.groupby('country_long').agg({'estimated_generation_gwh_2017': 'sum'})

,estimated_generation_gwh_2017
country_long,
Afghanistan,828.08
Albania,4608.91
Algeria,64568.81
Angola,3054.59
Antarctica,0.00
...,...
Vietnam,178034.30
Western Sahara,0.00
Yemen,2807.33


In [11]:
df.query('primary_fuel=="Other" and other_fuel1.notna()')[['primary_fuel', 'other_fuel1', 'other_fuel2', 'other_fuel3']]

,primary_fuel,other_fuel1,other_fuel2,other_fuel3
92,Other,Gas,NaN,NaN
277,Other,Gas,NaN,NaN
10202,Other,Gas,NaN,NaN
27522,Other,Gas,Biomass,NaN
28214,Other,Gas,NaN,NaN
28372,Other,Oil,NaN,NaN
29202,Other,Oil,NaN,NaN
32393,Other,Oil,NaN,NaN
32603,Other,Gas,NaN,NaN
33169,Other,Oil,Gas,NaN


In [12]:
df.primary_fuel.unique()

array(['Hydro', 'Solar', 'Gas', 'Other', 'Oil', 'Wind', 'Nuclear', 'Coal',
       'Waste', 'Biomass', 'Wave and Tidal', 'Petcoke', 'Geothermal',
       'Storage', 'Cogeneration'], dtype=object)

In [13]:
def energy_class(x):
    """Classify type of energy"""
    f_energy = ["Petcoke", "Gas", "Biomass", "Waste", "Coal", "Oil", "Other"]
    
    type = None
    if x in f_energy:
        type = "Fossil fuel"
    else:
        type = "Renewable fuel"
        
    return type

In [14]:
df['fuel_type'] = df['primary_fuel'].map(energy_class)

In [15]:
en_2017 = df.groupby('country_long').apply(lambda s: pd.Series({
    'fossil_energy_gwh_2017': s.query('fuel_type=="Fossil fuel"')[
        'estimated_generation_gwh_2017'].sum(),
    'fossil_energy_count_2017': s.query('fuel_type=="Fossil fuel"')[
        'estimated_generation_gwh_2017'].count(),
    'renewable_energy_gwh_2017': s.query('fuel_type=="Renewable fuel"')[
        'estimated_generation_gwh_2017'].sum(),
    'renewable_energy_count_2017': s.query('fuel_type=="Renewable fuel"')[
        'estimated_generation_gwh_2017'].count()
})).reset_index()

In [16]:
en_2017['fossil_2017_perc'] = round(en_2017['fossil_energy_gwh_2017']*100/en_2017['fossil_energy_gwh_2017'].sum(),2)
en_2017['renewable_2017_perc'] = round(en_2017['renewable_energy_gwh_2017']*100/en_2017['renewable_energy_gwh_2017'].sum(),2)   

In [18]:
# Output to notebook
output_notebook()

rc = Console()

# Map data
borders = 'mapping/ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp'
gdf = gpd.read_file(borders)[['ADMIN', 'ADM0_A3', 'geometry']]

# Rename columns
gdf.columns = ['country', 'country_code', 'geometry']

def get_minmax(data):
    """Get minimum and maximum for the scale"""
    en_max, en_min = 0, 0
    for i in ['fossil', 'renewable']:
        en_min = data[i+'_2017_perc'].min()
        en_max = data[i+'_2017_perc'].max()
    
    return en_max, en_min

def findcountry(country_name):
    """Find the official country name"""
    try:
        return pycountry.countries.get(name=country_name).official_name
    except:
        return country_name

class_radio = RadioButtonGroup(
    labels=["fossil", "renewable"],
    active=0
)

# - Function
def update_graph(attr, old, new):
    """Select energy class for dropdown"""
    class_name = classes[class_radio.active]
    
    if class_name == 'fossil':
        map_sc.data = d1[[name+'_energy_gwh_2017', name+'_2017_perc', name+'energy_count_2017']]
    
    # Minimum and Maximum
    en_max, en_min = get_minmax(en_2017)

    # Assign official names to data
    en_2017['country_long'] = en_2017['country_long'].apply(findcountry)

    gdf['country'] = gdf['country'].apply(findcountry)

    # Merge data with co-ordinates
    geo_df = gdf.merge(en_2017, left_on='country', right_on='country_long', how='left')

    # Fill Null values
    na_country = list(geo_df[geo_df['fossil_energy_gwh_2017'].isna()].country)

    geo_df.fillna(0, inplace=True)
    #geo_df['fo']
    # geo_df.dropna(axis=0, inplace=True)

    # Read data to json
    df_json = json.loads(geo_df.to_json())

    # Convert to string like object
    map_data = json.dumps(df_json)

    # Assign Source
    map_source = GeoJSONDataSource(geojson = map_data)

    # Map Geometry
    color_mapper = LinearColorMapper(palette=palette[::-1], low=en_min, high=en_max)

    # Colour scale
    tick_labels = {
        '2':'Index 2',
        '3':'Index 3',
        '4':'Index 4',
        '5':'Index 5',
        '6':'Index 6',
        '7':'Index 7',
        '8':'Index 8',
        '9':'Index 9',
        '10':'Index 10',
    }

    color_bar = ColorBar(
        color_mapper=color_mapper,
        label_standoff=5,
        width = 600, height = 30,
        border_line_color=None, 
        location = (20,0), 
        orientation = 'horizontal',
        major_label_overrides = tick_labels
    )

    # Map
    TOOLS = "pan,wheel_zoom,reset,hover,save"

    map_all = figure(plot_width=725, plot_height=500, 
                     title="Fossil fuel generation in GWh from different countries", 
                     tools=TOOLS, x_axis_location=None, y_axis_location=None, 
                     tooltips = [
                         ("Country", "@country"),
                         ("Energy generated %", "@fossil_2017_perc %"),
                         ("Energy generated", "@fossil_energy_gwh_2017{0,0.0} GWh"),
                         ("Plants", "@fossil_energy_count_2017")
                     ])

    map_all.grid.grid_line_color = None
    map_all.hover.point_policy = "follow_mouse"

    map_all.patches(
        "xs", "ys", source=map_source, 
        fill_color={
            "field": "fossil_2017_perc", 
            "transform": color_mapper
        }, 
        fill_alpha=0.7, line_color="black", line_width=0.5
    )

    map_all.add_layout(color_bar, 'below')
    map_all.add_layout(class_radio, 'above')

    rc.log(show(map_all), '[green]map charted')
    
update_graph()

Loading BokehJS ...

TypeError: update_graph() missing 3 required positional arguments: 'attr', 'old', and 'new'

In [ ]:
geo_df

In [ ]:
map_data